# Variational quantum eigensolver

This tutorial demonstrates a simplified variational quantum eigensolver (Peruzzo et al 2014). To showcase the hybrid principle of openqml, we first train a quantum circuit to minimize the energy expectation for a Hamiltonian $H$, 

$$ \langle \psi | H | \psi \rangle  = 0.1 \langle \psi_{w} | X_2 | \psi_w \rangle + 0.5 \langle \psi_w | Y_2 | \psi_w \rangle.  $$

Here, $|\psi_w\rangle$ is the state after applying the quantum circuit which depends on trainable weights $w =\{w_1, w_2\}$, and $X_2$, $Y_2$ denote the Pauli-X and Pauli-Y operator acting on the second qubit. 

We then turn things around and use a fixed quantum circuit to prepare $|\psi\rangle$, but train the coefficients of the Hamiltonian to minimize

$$ \langle \psi | H | \psi \rangle  = w_0 \langle \psi | X_2 | \psi \rangle + w_1 \langle \psi | Y_2 | \psi \rangle . $$


## Optimize the quantum circuit

Alongside the OpenQML framework, as well as the OpenQML and original NumPy library, we import the standard gradient descent optimizer.

In [1]:
import openqml as qm
from openqml import numpy as onp
import numpy as np
from openqml.optimize import GradientDescentOptimizer

22:58:57 WARNING No OpenQML configuration file found.


In [2]:
#dev = qm.device('projectq.simulator', wires=2)
dev = qm.device('default.qubit', wires=2)

### Quantum functions

The quantum circuit of the variational eigensolver is an ansatz that defines a manifold of possible quantum states. We use a "hardcoded" initial superposition, together with two rotations and a CNOT gate. 

In [3]:
def ansatz(weights):

    initial_state = np.array([1, 1, 0, 1])/np.sqrt(3)
    qm.QubitStateVector(initial_state, wires=[0, 1])

    qm.RX(weights[0], [0])
    qm.RY(weights[1], [1])
    qm.CNOT([0, 1])

A variational eigensolvers requires us to evaluate expectations of different Pauli operators. In this example, the Hamiltonian is expressed by only two single-qubit Pauli operators, namely the X and Y operator applied to the first qubit. 

Since these operators do not commute, we need two quantum functions, but they can reuse the same device we created. 

In [4]:
@qm.qfunc(dev)
def circuit_X(weights):
    ansatz(weights)
    return qm.expectation.PauliX(1)


@qm.qfunc(dev)
def circuit_Y(weights):
    ansatz(weights)
    return qm.expectation.PauliY(1)

### Objective

The objective of a VQE, usually called a "cost", is simply a linear combination of the expectations, which defines the expectation of the Hamiltonian we are interested in. 

In [5]:
def cost(weights):
    expX = circuit_X(weights)
    expY = circuit_Y(weights)
    return 0.1*expX + 0.5*expY

This cost defines the following landscape:
 <img src="figures/vqe_q_landscape.png" width="450"> 


### Optimization

We use the gradient descent optimizer from Tutorial 1.

In [6]:
weights0 = np.array([0., 0.])
print('Initial weights:', weights0)

o = GradientDescentOptimizer(0.5)
weights = weights0
for iteration in np.arange(1, 21):
    weights = o.step(cost, weights)
    print('Cost after step {:5d}: {: 0.7f}'.format(iteration, cost(weights)))
print('Optimized weights:', weights)

Initial weights: [ 0.  0.]
Cost after step     1:  0.0108032
Cost after step     2: -0.0428381
Cost after step     3: -0.0916198
Cost after step     4: -0.1337711
Cost after step     5: -0.1685817
Cost after step     6: -0.1962703
Cost after step     7: -0.2176645
Cost after step     8: -0.2338648
Cost after step     9: -0.2459913
Cost after step    10: -0.2550415
Cost after step    11: -0.2618342
Cost after step    12: -0.2670071
Cost after step    13: -0.2710399
Cost after step    14: -0.2742847
Cost after step    15: -0.2769957
Cost after step    16: -0.2793550
Cost after step    17: -0.2814913
Cost after step    18: -0.2834959
Cost after step    19: -0.2854327
Cost after step    20: -0.2873467
Optimized weights: [ 1.51493313  0.45108972]


 <img src="figures/vqe_q_landscape_gd.png" width="450"> 

## Optimizing the Hamiltonian coefficients

Instead of optimizing the circuit parameter, we can also use a fixed circuit,

In [8]:
def ansatz():

    initial_state = np.array([1, 1, 0, 1])/np.sqrt(3)
    qm.QubitStateVector(initial_state, wires=[0, 1])

    qm.RX(-0.5, [0])
    qm.RY( 0.5, [1])
    qm.CNOT([0, 1])
    
    
@qm.qfunc(dev)
def circuit_X():
    ansatz()
    return qm.expectation.PauliX(1)


@qm.qfunc(dev)
def circuit_Y():
    ansatz()
    return qm.expectation.PauliY(1)

and make the classical coefficients trainable.

In [9]:
def cost(weights):
    expX = circuit_X()
    expY = circuit_Y()
    return weights[0]*expX + weights[1]*expY

With this cost, ever smaller coefficients decrease the energy expectation.

In [10]:
weights0 = np.array([1., 1.])
print('Initial weights:', weights0)

o = GradientDescentOptimizer(0.5)
weights = weights0
for iteration in range(20):
    weights = o.step(cost, weights)
    print('Cost after step {:5d}: {: 0.7f}'.format(iteration+1, cost(weights)))
print('Optimized weights:', weights)

Initial weights: [ 1.  1.]
Cost after step     1:  0.6033687
Cost after step     2:  0.4618739
Cost after step     3:  0.3203791
Cost after step     4:  0.1788842
Cost after step     5:  0.0373894
Cost after step     6: -0.1041054
Cost after step     7: -0.2456002
Cost after step     8: -0.3870951
Cost after step     9: -0.5285899
Cost after step    10: -0.6700847
Cost after step    11: -0.8115795
Cost after step    12: -0.9530744
Cost after step    13: -1.0945692
Cost after step    14: -1.2360640
Cost after step    15: -1.3775588
Cost after step    16: -1.5190537
Cost after step    17: -1.6605485
Cost after step    18: -1.8020433
Cost after step    19: -1.9435381
Cost after step    20: -2.0850330
Optimized weights: [-3.25246528 -2.19617026]


Of course, the optimization landscape is nearly linear.

 <img src="figures/vqe_c_landscape_gd.png" width="450"> 

## Optimizing classical and quantum parameters

Of course, we can also optimize "classical" and "quantum" weights together by combining the two approaches from above.

In [11]:
def ansatz(weights):
    """ Ansatz of the variational circuit."""

    initial_state = np.array([1, 1, 0, 1])/np.sqrt(3)
    qm.QubitStateVector(initial_state, wires=[0, 1])

    qm.RX(weights[0], [0])
    qm.RY(weights[1], [1])
    qm.CNOT([0, 1])


@qm.qfunc(dev)
def circuit_X(weights):
    """Circuit measuring the X operator"""
    ansatz(weights)
    return qm.expectation.PauliX(1)


@qm.qfunc(dev)
def circuit_Y(weights):
    """Circuit measuring the Y operator"""
    ansatz(weights)
    return qm.expectation.PauliY(1)


def cost(weights):
    """Cost (error) function to be minimized."""

    expX = circuit_X(weights[0:2])
    expY = circuit_Y(weights[0:2])

    return weights[2]*expX + weights[3]*expY

weights0 = np.array([0., 0., 0., 0.])
print('Initial weights:', weights0)

o = GradientDescentOptimizer(0.5)
weights = weights0
for iteration in np.arange(1, 21):
    weights = o.step(cost, weights)
    print('Cost after step {:5d}: {: 0.7f}'.format(iteration, cost(weights)))
print('Optimized weights:', weights)

Initial weights: [ 0.  0.  0.  0.]
Cost after step     1: -0.2222222
Cost after step     2: -0.4560981
Cost after step     3: -0.7161481
Cost after step     4: -0.9971883
Cost after step     5: -1.2829652
Cost after step     6: -1.5645226
Cost after step     7: -1.8430111
Cost after step     8: -2.1208323
Cost after step     9: -2.3986104
Cost after step    10: -2.6763882
Cost after step    11: -2.9541660
Cost after step    12: -3.2319437
Cost after step    13: -3.5097215
Cost after step    14: -3.7874993
Cost after step    15: -4.0652771
Cost after step    16: -4.3430548
Cost after step    17: -4.6208326
Cost after step    18: -4.8986104
Cost after step    19: -5.1763882
Cost after step    20: -5.4541660
Optimized weights: [ 0.         -0.46364588 -7.31753151  0.        ]
